In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 데이터 불러오기

In [2]:
import numpy as np
import pandas as pd
file = '/content/drive/MyDrive/4조/데이터셋/음식_review.csv'
df = pd.read_csv(file)

## 번역 및 감성분석 함수화

In [3]:
import os
import sys
import urllib.request
import requests
import json

client_id = "mu5p1uj3j2"
client_secret = "rmz4IYQbITxiFn8bjUQ9V9ehr7oDkTo5tOUOJfs3"

def review_sentiment(review):

  encText = urllib.parse.quote(review)
  data = "source=en&target=ko&text=" + encText
  url = "https://naveropenapi.apigw.ntruss.com/nmt/v1/translation"
  request = urllib.request.Request(url)
  request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
  request.add_header("X-NCP-APIGW-API-KEY",client_secret)
  response = urllib.request.urlopen(request, data=data.encode("utf-8"))
  rescode = response.getcode()

  if(rescode==200):
      response_body = response.read()
      text_result = eval(response_body.decode('utf-8'))['message']['result']['translatedText']
      #print(text_result)
      #print(text)
      #print(response_body.decode('utf-8'))
  else:
      print("Error Code:" + rescode)




  url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"
  headers = {
      "X-NCP-APIGW-API-KEY-ID": client_id,
      "X-NCP-APIGW-API-KEY": client_secret,
      "Content-Type": "application/json"
  }
  content = text_result
  data = {
    "content": content
  }
  #print(json.dumps(data, indent=4, sort_keys=True))
  
  response = requests.post(url, data=json.dumps(data), headers=headers)
  rescode = response.status_code

  if(rescode == 200):
    #print (response.text)
    sen = eval(response.text)['document']['confidence']
    max_sen = max(sen, key = sen.get)
  else:
    print("Error : " + response.text)

  

  return  max_sen# 제일 큰 키 값찾기

In [4]:
#test
review_sentiment('i hate you'), review_sentiment('i love you')

('negative', 'positive')

## 적용

In [5]:
df.head(5)

,갈비구이,고등어구이,닭갈비,불고기,삼겹살,미역국,육개장,양념치킨,후라이드 치킨,배추김치,만두,라면,짜장면,짬뽕,잡채,김밥,비빔밥,떡볶이,제육볶음,보쌈,계란말이,파전,갈치조림,김치찌개,갈비찜,닭볶음탕,족발,삼계탕
0,I always marinade at least overnight when I do...,My family loved the sweet taste of the fish an...,닭갈비 is amazing so I've visited here 3 times!,I would recommend Bulgogi or Bibimbob. Everyon...,One of the nicest and authentic korean bbq ! E...,One of the healthiest Korean dishes to eat is ...,its probably my favorite soup in Korea. It is ...,"The chicken was so tender and crispy, it didn'...",Very good chicken and huge sharable portions. ...,"Writing as a non Korean, who loves kimchi but ...",We ordered the combo which comes with 2 mandoo...,The noodle was instant noodles but it has a ta...,The jajangmyeon was just ok per wife and her l...,My Korean exchange friends in college introduc...,Love this dish and so does the kiddos,The kimbap slices are large in size and tasty....,First time eating bibimbap and it tasted fresh...,The stew was very delicious with generous serv...,Spicy Stir-fried pork has always been my favor...,The food had me foaming at the mouth with its ...,This is one of my favorite side dishes.,"Haemul pajeon, aka Korean seafood pancake, is ...",I and my wife visited this restaurant for dinn...,My favorite Korean soup,"my family loved this dish, wasn’t able to use ...",My mouth is drooling,Jokbal wasn't that much as we expected. It tas...,"Chicken has been boiled soft rotten, the entra..."
1,As soon and the meat hit the grill and sizzled...,I ate two whole fish after cooked them. So good!,It was amazing! I wanna eat this everyday! Che...,Bulgogi (불고기) is classic Korean! It can be pre...,"Pork was tasted so good to chew into, with the...",when I gave birth to my children this wonderfu...,"Yukgaejang is a spicy beef and vegetable soup,...",The food here is good and the wings are nice a...,"First time being here, good location and not h...",If I can have crunchy Chinese pickled vegetabl...,These potstickers were a hit across the board!...,Had very nice beef ramen. I would have rated t...,We wanted to try the jajangmyeon and no doubt ...,This dish is heart warming and mouth watering :P,Many Koreans also make a vegetarian version by...,Love their big size kimbap. It is delicious an...,Bibimbap is one of my favorite dishes to order...,Wonderful spicy tteokbokki with oozing cheese ...,"Made this last night, and my neighbor said he ...","I've just spent 3 weeks in Korea, and tried ma...",I never knew an omelette could be so beautiful.,I remember makgeolli and pajeon is best,Good ingredients and skillful and great taste ...,so yum. my sisters and I loved it. we switched...,I actually love the sauce that I doubled it an...,Classic Korean chicken dish adapted for the pr...,Really yummy braised pork trotter that is cook...,Chicken soup is one of the specialities of Kor...
2,I would literally work for food if you were co...,I hope you are feeling better! Take care of yo...,I highly recommend it to all the trevelers! Or...,This has become a favorite go-to recipe at our...,It’s the best pork belly I ever had in Seoul. ...,Very different! I like it! My family did not l...,"Don't worry, yukgaejang is not as spicy as its...",Enjoyed ourselves pretty good with the deep fr...,Went there around 8pm and was full of people. ...,cheese smell decayed food but we can eat it. Y...,The mandus (dumpling) are very tasty. They hav...,It is delicious and really good for its price.,Tried the jajangmyeong and pork dumplings.. ta...,"When I ate this at my job in Atlanta, I fell i...",Soooooo... delicious and easy to make.,The kimbap are small and mostly filled with ri...,Bibimbap (pronounced bee-bim-BOP) is fun to sa...,Here we had our best Tteokbokki of our trip. L...,Really delicious!!!!! OMG best Korean food ever!!,"The bossam was soft, yet firm enough to hold i...",Used shallots as regular onion here is very st...,Omg I love Pajeon! Good tip on using cold,"If you swallow some bones, don't panic... just...",Delicious!! I m

In [ ]:
from tqdm import tqdm
review_dic = {}
for col in tqdm(df.columns):
  review_result = []
  for review in df[col].dropna():
    result = review_sentiment(review)
    review_result.append(result)
  review_dic[col] = review_result

100%|██████████| 28/28 [26:21<00:00, 56.48s/it]


In [ ]:
import pickle
with open('/content/drive/MyDrive/4조/데이터셋/review_result.pickle','wb') as fw:
    pickle.dump(review_dic, fw)

# load data
with open('/content/drive/MyDrive/4조/데이터셋/review_result.pickle', 'rb') as fr:
    loaded = pickle.load(fr)

In [ ]:
loaded

{'갈비구이': ['positive',
  'neutral',
  'neutral',
  'neutral',
  'positive',
  'negative',
  'neutral',
  'positive',
  'negative',
  'neutral',
  'neutral',
  'positive',
  'positive',
  'positive',
  'positive',
  'neutral',
  'positive',
  'positive'],
 '갈비찜': ['positive',
  'positive',
  'neutral',
  'neutral',
  'positive',
  'positive',
  'positive',
  'neutral',
  'positive',
  'positive',
  'neutral',
  'positive',
  'neutral',
  'positive',
  'neutral',
  'positive',
  'negative',
  'positive',
  'negative',
  'positive',
  'positive',
  'neutral',
  'neutral',
  'negative',
  'positive',
  'neutral',
  'neutral'],
 '갈치조림': ['positive',
  'positive',
  'negative',
  'positive',
  'positive',
  'positive',
  'positive',
  'positive',
  'negative',
  'positive',
  'positive',
  'neutral',
  'positive',
  'positive',
  'negative',
  'positive',
  'positive',
  'positive',
  'positive',
  'positive',
  'neutral',
  'positive',
  'neutral',
  'positive',
  'positive',
  'positive',
 